In [1]:
import glob
import os
import json
import pandas as pd
from tqdm.auto import tqdm

import numpy as np
from CotModel import CotModel
from LincModel import LincModel
from NaiveModel import NaiveModel

/root/miniconda3/envs/linc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
glob.glob(os.path.join("results/folio", '*.json'))

['results/folio/cot - Llama-2-13b-chat-hf (2024-11-29_06-45-56).json',
 'results/folio/cot - Llama-2-7b-chat-hf (2024-11-29_03-23-47).json',
 'results/folio/cot - Llama-3.1-8B-Instruct (2024-11-28_23-55-42).json',
 'results/folio/cot - Llama-3.2-1B-Instruct (2024-11-28_22-34-15).json',
 'results/folio/cot - Llama-3.2-3B-Instruct (2024-11-28_23-11-19).json',
 'results/folio/linc - Llama-2-13b-chat-hf (2024-11-29_13-34-05).json',
 'results/folio/linc - Llama-2-7b-chat-hf (2024-11-29_05-02-07).json',
 'results/folio/linc - Llama-3.1-8B-Instruct (2024-11-29_02-02-06).json',
 'results/folio/linc - Llama-3.2-1B-Instruct (2024-11-29_00-51-35).json',
 'results/folio/linc - Llama-3.2-3B-Instruct (2024-11-29_00-17-26).json',
 'results/folio/naive - Llama-2-13b-chat-hf (2024-11-28_23-06-36).json',
 'results/folio/naive - Llama-2-7b-chat-hf (2024-11-28_22-07-31).json',
 'results/folio/naive - Llama-3.1-8B-Instruct (2024-11-28_21-55-17).json',
 'results/folio/naive - Llama-3.2-1B-Instruct (2024-11-

In [ ]:
data_path = "data/folio.csv"
paths = glob.glob(os.path.join("results/folio", '*.json'))

def parse_cot(row):
    answers = [CotModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

def parse_linc(row):
    answers = [LincModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

def parse_naive(row):
    answers = [NaiveModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

results_df_dict = {}

for path in tqdm(paths):
    with open(path, "r") as file:
        results = json.load(file)
        
    results_df = pd.DataFrame(results)
    results_df["label"] = pd.read_csv(data_path)["label"]

    if "cot" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_cot(row)), axis=1)
    elif "linc" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_linc(row)), axis=1)
    elif "naive" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_naive(row)), axis=1)
    results_df_dict[path.split("/")[-1]] = results_df


100%|██████████| 15/15 [15:17<00:00, 61.19s/it] 


In [ ]:
results_df_list

In [4]:
import pickle

with open("data/results_df_list.pkl", "wb") as file:
    pickle.dump(results_df_list, file)

In [3]:
with open("results/folio/linc - Llama-2-7b-chat-hf (2024-11-29_05-02-07).json", "r") as file:
    results = json.load(file)

data_path = "data/folio.csv"
results_df = pd.DataFrame(results)
results_df["label"] = pd.read_csv(data_path)["label"]

In [ ]:
import re
import numpy as np

def parse_linc(row):
    answers = [LincModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return accuracies, mean
    
results_df[["accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_linc(row)), axis=1)

['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'True', 'Uncertain', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Uncertain', 'Uncertain', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Error']
['Uncertain', 'Uncertain', 'Uncertain', 'Uncertain', 'Uncertain']
['Uncertain', 'Uncertain', 'Uncertain', 'Uncertain', 'Uncertain']
['Uncertain', 'Uncertain', 'Uncertain', 'Uncertain', 'Uncertain']
['True', 'Error', 'Uncertain', 'True', 'True']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Error', 'Error', 'Uncertain']
['Error', 'Uncertain', 'Error', 'Uncertain', 'Uncertain']
['Error', 'Error', 'Error', 'Error', 'Error']
['Error', 'Error', 'Uncertain', 'Error', 'Uncertain']
['True', 'Uncertain', 'Error',

KeyboardInterrupt: 

In [5]:
response = results_df.iloc[2].responses[0][3]
print(response)

  <EVALUATE>

TEXT: All employees who schedule a meeting with their customers will appear in the company today.
FOL: all x. (ScheduleMeeting(x) -> AppearInCompany(x))

TEXT: Everyone who has lunch in the company schedules meetings with their customers.
FOL: all x. (HaveLunchInCompany(x) -> ScheduleMeetings(x))

TEXT: Employees will either have lunch in the company or have lunch at home.
FOL: all x. (HaveLunchInCompany(x) | HaveLunchAtHome(x))

TEXT: If an employee has lunch at home, then he/she is working remotely from home.
FOL: all x. (HaveLunchAtHome(x) -> WorkingRemotelyFromHome(x))

TEXT: All employees who are in other countries work remotely from home.
FOL: all x. (InOtherCountry(x) -> WorkingRemotelyFromHome(x))

TEXT: No managers work remotely from home.
FOL: all x. (Manager(x) -> -WorkingRemotelyFromHome(x))

TEXT: James is either a manager and appears in the company today or neither a manager nor appears in the company today.
FOL: all x. (Manager(james) & AppearInCompanyToday

In [6]:
strings = LincModel._extract_fol_strings(response)
strings

['all x. (ScheduleMeeting(x) -> AppearInCompany(x))',
 'all x. (HaveLunchInCompany(x) -> ScheduleMeetings(x))',
 'all x. (HaveLunchInCompany(x) | HaveLunchAtHome(x))',
 'all x. (HaveLunchAtHome(x) -> WorkingRemotelyFromHome(x))',
 'all x. (InOtherCountry(x) -> WorkingRemotelyFromHome(x))',
 'all x. (Manager(x) -> -WorkingRemotelyFromHome(x))',
 'all x. (Manager(james) & AppearInCompanyToday(james) | -Manager(james) & -AppearInCompanyToday(james))']

In [7]:
LincModel._convert_fol_exps(strings[:-1], strings[-1])

([<AllExpression all x.(ScheduleMeeting(x) -> AppearInCompany(x))>,
  <AllExpression all x.(HaveLunchInCompany(x) -> ScheduleMeetings(x))>,
  <AllExpression all x.(HaveLunchInCompany(x) | HaveLunchAtHome(x))>,
  <AllExpression all x.(HaveLunchAtHome(x) -> WorkingRemotelyFromHome(x))>,
  <AllExpression all x.(InOtherCountry(x) -> WorkingRemotelyFromHome(x))>,
  <AllExpression all x.(Manager(x) -> -WorkingRemotelyFromHome(x))>],
 <AllExpression all x.((Manager(james) & AppearInCompanyToday(james)) | (-Manager(james) & -AppearInCompanyToday(james)))>,
 <NegatedExpression -all x.((Manager(james) & AppearInCompanyToday(james)) | (-Manager(james) & -AppearInCompanyToday(james)))>)

In [8]:
from nltk.inference.prover9 import Prover9
fol_exps_premises, fol_exp_conclusion, fol_exp_not_conclusion = LincModel._convert_fol_exps(strings[:-1], strings[-1])
prover = Prover9()
result = prover.prove(fol_exp_conclusion, fol_exps_premises)
notResult = prover.prove(fol_exp_not_conclusion, fol_exps_premises)

In [9]:
result

False

In [10]:
notResult

False

In [ ]:
LincModel

['  <EVALUATE>\n\nTEXT: If people perform in school talent shows often, then they attend and are very engaged with school events.\nFOL: all x. (PerformInSchoolTalentShowsOften(x) -> AttendAndEngagedWithSchoolEvents(x))\n\nTEXT: People either perform in school talent shows often or are inactive and disinterested members of their community.\nFOL: all x. (PerformInSchoolTalentShowsOften(x) -> (-InactiveAndDisinterestedMemberOfTheirCommunity(x)) | (InactiveAndDisinterestedMemberOfTheirCommunity(x)))\n\nTEXT: If people chaperone high school dances, then they are not students who attend the school.\nFOL: all x. (ChaperoneHighSchoolDances(x) -> NotStudentWhoAttendsTheSchool(x))\n\nTEXT: All people who are inactive and disinterested members of their community chaperone high school dances.\nFOL: all x. (InactiveAndDisinterestedMemberOfTheirCommunity(x) -> ChaperoneHighSchoolDances(x))\n\nTEXT: All young children and teenagers who wish to further their academic careers and educational opportunit

In [ ]:
results_df

,index,responses,label,accuracies,mean
0,0,"[ Sure, I'd be happy to help! Here's my evalu...",Uncertain,"[0, 0, 0, 0, 1]",0.2
1,1,"[ Sure, I'd be happy to help! Let's think ste...",True,"[0, 1, 0, 1, 1]",0.6
2,2,"[ Sure, I'd be happy to help! Let's think ste...",Uncertain,"[0, 0, 0, 0, 0]",0.0
3,3,"[ Sure, I'd be happy to help! Here's my evalu...",Uncertain,"[0, 1, 0, 0, 0]",0.2
4,4,"[ Sure, I'd be happy to help! Here's my evalu...",True,"[1, 0, 0, 0, 0]",0.2
...,...,...,...,...,...
177,177,"[ Sure, I'd be happy to help! Here's my evalu...",False,"[1, 0, 0, 0, 1]",0.4
178,178,"[ Sure, I'd be happy to help! Here's my evalu...",False,"[0, 1, 1, 0, 0]",0.4
179,179,"[ Sure, I'd be happy to help! Let's think ste...",False,"[0, 1, 0, 1, 1]",0.6
180,180,"[ Sure, I'd be happy to help! Here's my evalu...",Uncertain,"[0, 1, 1, 1, 0]",0.6
